In [5]:
import os
import rasterio
from rasterio.warp import reproject, Resampling
from rasterio.enums import Resampling
from rasterio.transform import from_origin

In [4]:
filepath = "combined/4km_out_test.tif"

# Define the target bounding box and resolution
min_lon, max_lon = -124.409591, -114.131211
min_lat, max_lat = 32.534156, 42.009518
target_crs = 'EPSG:4269'
target_width, target_height = 1000, 1000
target_resolution = (0.01, 0.01)

def transform_tif(input_path, output_path):
    with rasterio.open(input_path) as src:
        # Calculate the target transform
        transform = from_origin(min_lon, max_lat, target_resolution[0], target_resolution[1])

        # Define the metadata for the output file
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': target_width,
            'height': target_height,
            'driver': 'GTiff'
        })

        # Create the output file with the updated metadata
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest  # or Resampling.nearest if the data is categorical
                )

with rasterio.open(filepath) as src:
    # Get resolution
    resolution = src.res
    # Get bounding box (min/max longitude and latitude)
    bounds = src.bounds
    min_lon, max_lon = bounds.left, bounds.right
    min_lat, max_lat = bounds.bottom, bounds.top
    # Get pixel dimensions
    width, height = src.width, src.height
    # Print information
    print(f"File: {filepath}")
    print(f"  Resolution: {resolution}")
    print(f"  Bounding Box: ({min_lon}, {max_lon}, {min_lat}, {max_lat})")
    print(f"  Pixel Dimensions: {width} x {height}")
    print()

File: combined/4km_out_test.tif
  Resolution: (0.01, 0.01)
  Bounding Box: (-124.40970717799996, -114.13970717799997, 32.539513750000054, 42.00951375000005)
  Pixel Dimensions: 1027 x 947

